### Multi-agent simulator for evaluating chatbots

Creation of a 'virtual user' to simulate a conversation. Persona that is passed in the input is given to the simulated user and the initial question goes the the agent to get an output. Agent and simulated user goes through two-way prompting until the simulated user is satisfied.

![multi agent collaboration flowchart](images/agent-sim.png)

## Define agent (chatbot)

In the following cell we define the logic for a new agent which is the chatbot for customer service. We provide a system message as the persona to guide the llm on how it is supposed to act and take in a messsage from the user.

In [133]:
from typing import List
from rich.pretty import pprint as print
import openai

def my_chat_bot(messages: List[dict]) -> dict:
    """
    create chatbot node that has the starting prompt defining the persona
    """
    system_message = {
        "role": "system",
        "content": "You are a customer support agent for an airline.",
    }
    messages = [system_message] + messages
    completion = openai.chat.completions.create(
        messages=messages,model="gpt-3.5-turbo"
    )
    return completion.choices[0].message.model_dump()

In [134]:
my_chat_bot([{"role": "user", "content": "hi!"}])

{'content': 'Hello! Welcome to our customer support service. How can I assist you today?',
 'refusal': None,
 'role': 'assistant',
 'audio': None,
 'function_call': None,
 'tool_calls': None}

## Define simulated user

In the following cell we define the logic for a simulated user and provide a predefined persona. Using chat prompt template, we provide 2 placeholders: instructions and messages. instructions is to give more personalised context for the system prompt and messages is the placeholder for the list of base messages that contains all the states and messages.

In [135]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

system_prompt_template = """You are a reasonable customer of an airline company. \
You are interacting with customer support of the airline company. \

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
instructions = """Your name is Harrison. You are trying to get a full refund for the trip you took to Alaska. \
You are to answer any questions that the customer support has and try to negotiate where possible \
This trip happened 5 years ago."""

prompt = prompt.partial(name="Harrison", instructions=instructions)

model = ChatOpenAI()

simulated_user = prompt | model

In [136]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Hello! How can I assist you today?")]
print(simulated_user.invoke({"messages": messages}))

AIMessage(
│   content='Hello, I would like to inquire about getting a full refund for a trip I took to Alaska with your airline company five years ago.',
│   additional_kwargs={'refusal': None},
│   response_metadata={
│   │   'token_usage': {
│   │   │   'completion_tokens': 28,
│   │   │   'prompt_tokens': 105,
│   │   │   'total_tokens': 133,
│   │   │   'completion_tokens_details': {
│   │   │   │   'accepted_prediction_tokens': 0,
│   │   │   │   'audio_tokens': 0,
│   │   │   │   'reasoning_tokens': 0,
│   │   │   │   'rejected_prediction_tokens': 0
│   │   │   },
│   │   │   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
│   │   },
│   │   'model_name': 'gpt-3.5-turbo-0125',
│   │   'system_fingerprint': None,
│   │   'finish_reason': 'stop',
│   │   'logprobs': None
│   },
│   id='run-8233d9de-fb82-4a72-ba25-34ec62952471-0',
│   usage_metadata={
│   │   'input_tokens': 105,
│   │   'output_tokens': 28,
│   │   'total_tokens': 133,
│   │   'input_token_details': {'audio': 0, 'cache_read': 0},
│   │   'output_token_details': {'audio': 0, 'reasoning': 0}
│   }
)

## Defining nodes

In the following cells, we create the nodes for the chatbot and the simulated user respectively. All the nodes should take in a list of messages and return a list of messages to be added to the state.

note: one tricky thing here is which messgaes are which, since the roles are reversed in the chatbot and simulated user nodes. To solve this, we define a new method called _swap_roles, which changes all human messages to ai messages and vice versa. By following this workaround, inputs to the node will always be taken as a human message, and the repsonse of the llm will be the ai message.

### chat_bot_node

In [142]:
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage

def chat_bot_node(state):
    messages = state["messages"]
    #  convert from langchhain format to OpenAI format, whcih is what the chatbot function expects
    messages = [convert_message_to_dict(m) for m in messages]

    # invoking chatbot with converted messages
    chat_bot_response = my_chat_bot(messages)
    
    # return with the response of the chatbot agent as an ai message
    return {"messages": [AIMessage(content=chat_bot_response["content"])]}

## simulated_user_node

In [143]:
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages

def simulated_user_node(state):
    messages = state["messages"]
    # swap roles of messages to better fit simulated_user logic
    new_messages = _swap_roles(messages)
    # invoking simulated_user with converted messages
    simulated_user_response = simulated_user.invoke({"messages": new_messages})
    # the simulated_user_response is an ai message, so it will need to be converted to a human message
    return {"messages": [HumanMessage(content=simulated_user_response.content)]}

## Defining edges

After invoking the simulated user, there are two outcomes:

1) continue and call the customer support bot
2) finish and the conversation is over

for point 2, we use the defined stop keyword FINISHED in the system_prompt or if the conversation is more than 6 messages long (arbitrary number)

In [144]:
def should_continue(state):
    messages = state["messages"]
    if len(messages) > 10:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

## Define graph

In [145]:
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)

# force every response from chatbot to the simulated user by only adding an edge from chatbot to simulated user
graph_builder.add_edge("chat_bot", "user")
# add a conditional edge after invoking user node, to either go to END or back to the chat_bot node based on the should_continue condition
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # based on the response of should_continue (which is based on the repsonse of the user node), we will move to either END or back to the chat_bot node
    {
        "end": END,
        "continue": "chat_bot",
    }
)

# define the entrypoint to the graph
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

## Testing simulation

In [147]:
for chunk in simulation.stream({"messages": []}):
    if END not in chunk:
        print(chunk)
        print("----")

{
│   'chat_bot': {
│   │   'messages': [
│   │   │   AIMessage(
│   │   │   │   content='Hello! How can I assist you today with your travel needs?',
│   │   │   │   additional_kwargs={},
│   │   │   │   response_metadata={},
│   │   │   │   id='a6a4e415-bcf4-483e-8201-369d41d9ebed'
│   │   │   )
│   │   ]
│   }
}

'----'

{
│   'user': {
│   │   'messages': [
│   │   │   HumanMessage(
│   │   │   │   content='Hello, I am Harrison. I am looking to inquire about the possibility of getting a full refund for a trip I took to Alaska with your airline company five years ago.',
│   │   │   │   additional_kwargs={},
│   │   │   │   response_metadata={},
│   │   │   │   id='de7bcf70-c7a8-4051-a986-83732b2cc8f4'
│   │   │   )
│   │   ]
│   }
}

'----'

{
│   'chat_bot': {
│   │   'messages': [
│   │   │   AIMessage(
│   │   │   │   content='Hello Harrison! I can certainly look into that for you. Can you please provide me with your booking information or any relevant details about the trip to Alaska? This will help me to further assist you with your refund request.',
│   │   │   │   additional_kwargs={},
│   │   │   │   response_metadata={},
│   │   │   │   id='d5a9a4b9-b1cc-4bf8-89fb-e6b21202f112'
│   │   │   )
│   │   ]
│   }
}

...

'----'

{
│   'chat_bot': {
│   │   'messages': [
│   │   │   AIMessage(
│   │   │   │   content="Thank you for confirming the payment method and providing more information, Harrison. I will go ahead and initiate the refund process for your trip to Alaska in June 2016. Please allow some time for the refund to be processed, as it may take a few business days to reflect in your account, depending on the credit card issuer's processing time. Is there anything else I can assist you with today?",
│   │   │   │   additional_kwargs={},
│   │   │   │   response_metadata={},
│   │   │   │   id='e26fc93d-7fbd-4d4d-9d27-33419a261a4f'
│   │   │   )
│   │   ]
│   }
}

'----'

{
│   'user': {
│   │   'messages': [
│   │   │   HumanMessage(
│   │   │   │   content='I appreciate your help in initiating the refund process for my trip to Alaska. Thank you for your assistance. If there are no further steps required from my end, then I have no additional requests at this time. Thank you again for your help. \n\nFINISHED',
│   │   │   │   additional_kwargs={},
│   │   │   │   response_metadata={},
│   │   │   │   id='397ae06a-857d-4311-abee-2923ae9b3459'
│   │   │   )
│   │   ]
│   }
}

'----'

This workflow demonstrates the possibility of using agent simulators to test out tools. By adopting different personas in the agent simulator, testing of tools can become more robust and provide more coverage.